In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from dataloader import trainloader
from models.base_hovernet import targets


import os
import argparse
import importlib
from glob import glob
from tqdm import tqdm

from config import normal_Config, uniform_Config

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dataloader import trainloader
from models.base_hovernet import targets

In [3]:
data = normal_Config['shape_info'][normal_Config['run_mode']]
data

{'input_shape': [270, 270], 'mask_shape': [80, 80]}

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
# Set random seed
torch.manual_seed(normal_Config['seed'])
np.random.seed(normal_Config['seed'])
# Load dataset
print("Loading dataset...")

run_mode  = normal_Config['run_mode']
input_dataset = trainloader.FileLoader(
    glob(os.path.join(normal_Config['dataset_path'], '*.npy')),     
    mode=normal_Config['run_mode'],
    with_type=normal_Config['with_type'],
    setup_augmentor=True, # 이거 True / False 차이 알아보기
    target_gen=[targets.gen_targets, {}],
    **normal_Config['shape_info'][normal_Config['run_mode']]
)
dataloader = DataLoader(
    input_dataset,
    num_workers=2,
    batch_size=normal_Config['batch_size'],
    shuffle=normal_Config['run_mode'] == "train",
    drop_last=normal_Config['run_mode'] == "train",
    # worker_init_fn=worker_init_fn, # TODO: multiGPU에서 
)

Using device: cuda
Loading dataset...


In [13]:
# dict_keys(['img', 'tp_map', 'hv_map', 'np_map'])
print(len(input_dataset))
indices = [i for i in range(len(input_dataset))]  # 확인하고 싶은 index 리스트
unique_values = np.unique(np.concatenate([input_dataset[i]['tp_map'].flatten() for i in indices]))
print(unique_values)


378


/mnt/Hover-Net/Hover-Net_Custom/models/base_hovernet/targets.py:33: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  crop_ann = morph.remove_small_objects(crop_ann, min_size=30)


[0 1 2 3 4 5 6 7]


In [ ]:
module = importlib.import_module(
    "models.%s.opt" % normal_Config['model_name']
)
model_config = module.get_config(normal_Config['nr_type'], normal_Config['model_mode'])

In [20]:
from models.base_hovernet.opt import get_config

temp = get_config(8, 'train')
for i in range(len(temp['phase_list'])):
    print(temp['phase_list'][i]['run_info'])

{'net': {'desc': <function get_config.<locals>.<lambda> at 0x7f80a4075b80>, 'optimizer': [<class 'torch.optim.adam.Adam'>, {'lr': 0.0001, 'betas': (0.9, 0.999)}], 'lr_scheduler': <function get_config.<locals>.<lambda> at 0x7f80a4075ca0>, 'extra_info': {'loss': {'np': {'bce': 1, 'dice': 1}, 'hv': {'mse': 1, 'msge': 1}, 'tp': {'bce': 1, 'dice': 1}}}, 'pretrained': './pretrained/ImageNet-ResNet50-Preact_pytorch.tar'}}
{'net': {'desc': <function get_config.<locals>.<lambda> at 0x7f80e0145550>, 'optimizer': [<class 'torch.optim.adam.Adam'>, {'lr': 0.0001, 'betas': (0.9, 0.999)}], 'lr_scheduler': <function get_config.<locals>.<lambda> at 0x7f80e0145430>, 'extra_info': {'loss': {'np': {'bce': 1, 'dice': 1}, 'hv': {'mse': 1, 'msge': 1}, 'tp': {'bce': 1, 'dice': 1}}}, 'pretrained': -1}}


In [ ]:
from models.base_hovernet.utils import crop_to_shape, dice_loss, mse_loss, msge_loss, xentropy_loss

In [ ]:
def train_step(batch_data)
    # Move data to device
    loss_